In [1]:
import numpy as np

### Symbolic Differentiation
Symbolic differentiation is a form of programatically expressing the derivative by transforming an expression using known rules for differentiation.  For example, given a composed function $f(g(x),h(x)) = g(x) + h(x)$, the derivative can be determined symbolically using the sum rule.

\begin{align}
f(g(x),h(x)) &= g(x) + h(x) \\
\cfrac{\partial}{\partial x} f(g(x),h(x)) &= \cfrac{\partial}{\partial x} (g(x) + h(x)) \\
&= \cfrac{\partial}{\partial x} g(x) + \cfrac{\partial}{\partial x} h(x) \\
\end{align}

Below, a test function is given along with a function of its derivative determined manually and written symbolically.  You can try evaluating both for various values of x and y.

In [2]:
def f(x,y):
    return x*y + x*x + y
#The parameter index indiates which variable the derivative is taken with respect to. 0 indicates x and 1 indicates y.
def f_prime(x,y,index):
    if index == 0: return y + 2 * x
    else: return x + 1

In [3]:
x = 3
y = 2
print("Primal evaluation of function:",f(x,y))
print("Derivative with respect to x:",f_prime(x,y,0))
print("Derivative with respect to y:",f_prime(x,y,1))

Primal evaluation of function: 17
Derivative with respect to x: 8
Derivative with respect to y: 4


While this may appear striaghtforward and an easy way to determine the derivative issues often arise with larger expressions, which may have derivatives which are exponentially increasing in size.  Take the example given in class of a function $f(x)$.

$$f(x) = \cfrac{cos(ln(x))}{sin(ln(x)^{-1})}$$

The primal evaluation of this expression appears relatively simple, containing only five operators for which it is necessary to evaluate only once (i.e. $sin()$, $cos()$, $ln(x)$, $ln(x)^{-1}$, and division).  However, the derivative of this function is as follows:)

$$f'(x) = \cfrac{csc(ln(x)^{-1})(cos(ln(x))cot(ln(x)^{-1}) - ln(x)^2sin(ln(x)))}{x(ln(x)^{2})}$$

Not only does this derivative contain more operators than the primal expression (e.g. $csc()$ and $cot()$), but, assuming no caching and that powers are a seperate operation, calculating the deriavtive requires 20 seperate operator evaluations.  This is a four fold increase from the primal evaluation!  In general, the derivatives of functions may increase exponentially, leading to exponential run times.  As well, symbolic differentiation is poorly suited to handle complex programatic cases.  For example, what happens if we want to take the derivative of a function that contains recursion or a while loop?

### Numerical Differentiation (Finite Differences)
Another method of programatic differentiation is numerical differentation, the specific form of which referenced here is finite differences.  Rather than symbolic differentiation, numerical differentiation is a discretizatized approximation of the derivative, and thus will not lead to an exact solution, merely a close solution.

The equation for the finite difference can be arrived at from the limit definition of the derivative.

$$f'(x) = lim_{h \to 0} \cfrac{f(x + h) - f(x)}{h}$$

However, instead of the infitesimal, for finite differences the value of $h$ is simply a small number used to approximate this infitesimal.  Thus, the above equation can be reduced for such an $h$.

$$f'(x) = \cfrac{f(x + h) - f(x)}{h} + \mathcal{O}(h)$$

Below a function is defined to calculate this finite difference form, which is commonly termed the forward difference.  The parameter $f$ takes in a function, $h$ takes in a small float to represent an infitesimal, params are a list of parameters values for the function, and index is the index for which to take the derivative with respect to.  As above, try evaluating for various values of $x$, $y$, and $h$.

#### Forward Difference

In [12]:
def forward_difference(f,h,params,index):
    param_h = params.copy()
    param_h[index] += h
    return (f(*param_h) - f(*params))/h

In [13]:
x = 3
y = 2
h = 1e-3
print("Primal evaluation of function:",f(x,y))
print("Derivative with respect to x:",forward_difference(f,h,[x,y],0))
print("Derivative with respect to y:",forward_difference(f,h,[x,y],1))

Primal evaluation of function: 17
Derivative with respect to x: 8.001000000000147
Derivative with respect to y: 4.000000000001336


#### Centered Difference
However, consider that the derivative of a function does not necessarily need to be approximated with a point slightly ahead of it (i.e. $f(x + h)$).  It may also be approximated with the evaluation of a point slightly behind it, known as the backward difference, or with the evaluation of two points slightly behind and ahead of it, known as a centered difference.  The centered difference is more accurate than the forward difference, but requires an extra function evaluation:
$$
f'(x) = \frac{f(x+h) - f(x-h)}{2h} + \mathcal{O}(h^2)
$$
**Here, fill out the centered difference method below and contrast your test results with the forward difference method above.**

In [16]:
def centered_difference(f,h,params,index):
    '''
    TODO: Implement the centered difference, should not be significant different than forward difference
    '''
#     params_h = params.copy()
    params_h1 = params.copy()
    params_h2 = params.copy()
    params_h1[index] += h
    params_h2[index] -= h
    return (f(*params_h1) - f(*params_h2))/(2*h)
    

In [17]:
x = 3
y = 2
h = 1e-3
print("Primal evaluation of function:",f(x,y))
print("Derivative with respect to x:",centered_difference(f,h,[x,y],0))
print("Derivative with respect to y:",centered_difference(f,h,[x,y],1))

Primal evaluation of function: 17
Derivative with respect to x: 7.999999999999119
Derivative with respect to y: 4.000000000001336


While numerical differentation may appear to escape many of the pitfalls of symbolic differentiation, note that to calculate the gradient, the function will need to be evaluated twice for each input.  Thus the gradient must be determined in $2n$ or generally $\mathcal{O}(n)$ evaluations of the function where $n$ is the number of inputs.

### Automatic Differentiation
Contrasting both numerical and symbolic differentiation is automatic differentiation.  The basic idea behind automatic differentiation is that at the elemental level derivatives are determined symbolically.  For example, the differential values of the operators $\texttt{+}$, $\texttt{-}$, $\texttt{*}$, and $\texttt{/}$ may be noted and stored explicitly in the program.  Therefore, whenever those operations are encountered the program will know the value of the derivative and thus be able to calculate it.  

By breaking a function into an elemental form as such, it can be conceptualized as a graph.  Here refer to Figure 4 in the automatic differentiation paper, which shows the graph for a sample function $f(x_1,x_2) = ln(x_1) + x_1x_2 - sin(x_2)$.  Each node is a specific value, and each edge is an operation taking the node to a new value.  Conceptually these operations need to be stored as the graph is created, with a data structure to handle the edges containing the operations and nodes containing the values.

#### Forward Mode 

The forward mode of automatic differentiation is the most simple of the two discussed in the paper, and thus will be presented first here.  In contrast to the reverse mode, it is not necessary to explictly create the graph, rather it can be implicitly walked through by using a specific data structure, dual numbers.

Dual numbers are defined somewhat similarly to imaginary numbers by using the construction $x + x'\epsilon$ where rather than $-1$, $\epsilon^2 = 0$ (note that this can just be viewed as a truncated Taylor series).  Conceptually, this means that any values of $\epsilon$ with an order greater than 2 are $0$, and thus a dual number can also be considered to be a truncated taylor series.  The power in using dual numbers comes from their property when the elemental operations are applied to them.  Consider the result where addition, subtraction, and multiplication are applied to two dual numbers.

$$(x_1 + x'_1\epsilon) + (x_2 + x'_2\epsilon) = (x_1 + x_2) + (x'_1 + x'_2)\epsilon$$
$$(x_1 + x'_1\epsilon) - (x_2 + x'_2\epsilon) = (x_1 - x_2) + (x'_1 - x'_2)\epsilon$$
$$(x_1 + x'_1\epsilon)(x_2 + x'_2\epsilon) = (x_1 x_2)+(x_1x'_2 + x_2x'_1)\epsilon+(x'_1x'_2)\epsilon^2 = (x_1x_2)+(x_1x'_2 + x_2x'_1)\epsilon$$

The results of the addition and multiplication operations on dual numbers thus perfectly mirror the sum and product rules for derivatives.

In terms of implementation, dual numbers may be represented as a class wrapping two floats, one to represent the primal value and one to represent the derivative.  The elemental operations can then be overloaded to correctly calculate the result from operating on dual numbers.  An implementation of this type is presented below.

In [18]:
class Dual():
    #initialize the data structure so that it has values x and x'
    def __init__(self,val,deriv):
        self.val = val
        self.deriv = deriv
    #create a string representation
    def __str__(self):
        return "Value: " + str(self.val) + " Derivative: " + str(self.deriv)
    def __repr__(self):
        return str(self)
    #overload the + operator
    def __add__(self,other):
        return Dual(self.val+other.val,self.deriv+other.deriv)
    #overload the - operator
    def __sub__(self,other):
        return Dual(self.val-other.val,self.deriv-other.deriv)
    #overload the * operator
    def __mul__(self,other):
        return Dual(self.val*other.val,self.val*other.deriv+other.val*self.deriv)
    #the negative operator simply takes the negative of both the derivative and primal value
    def __neg__(self):
        return Dual(-self.val,-self.deriv)

As above, this method of derivation can be tested with a sample function by simply passing Dual() objection into a function containing only the overloaded operators.  Note here however, that this means that an index parameter cannot be passed to the function calculating the derivative since the primal function is unchanged, only the data structure is modified.  The initial derivative value for the dual numbers passed to the function will be 1 for the variable of interest (since $\cfrac{\partial x_i}{\partial x_i} = 1$) and 0 for all other variables (since $\cfrac{\partial x_i}{\partial x_j} = 0$ for $i \neq j$).

In [19]:
x = 3
y = 2
print("Primal evaluation of function:",f(x,y))
print("Derivative with respect to x:",f(Dual(x,1),Dual(y,0)))
print("Derivative with respect to y:",f(Dual(x,0),Dual(y,1)))

Primal evaluation of function: 17
Derivative with respect to x: Value: 17 Derivative: 8
Derivative with respect to y: Value: 17 Derivative: 4


However, it should be noted that the above operators are not the only ones which can be implemented using dual numbers.  Any function with a valid derivative can be implemented as a dual number.  **As a first exercise overload the float division operator using the above Dual() class and test it on a function containing a division operation.** *Bonus: the paper discusses the means of implementing more complex functions like $\sin(x)$ or $\ln(x)$.  Implement such a function (of your choice) for dual numbers and verify its correctness.*

In [29]:
class Dual():
    #initialize the data structure so that it has values x and x'
    def __init__(self,val,deriv):
        self.val = val
        self.deriv = deriv
    #create a string representation
    def __str__(self):
        return "Value: " + str(self.val) + " Derivative: " + str(self.deriv)
    def __repr__(self):
        return str(self)
    #overload the + operator
    def __add__(self,other):
        return Dual(self.val+other.val,self.deriv+other.deriv)
    #overload the - operator
    def __sub__(self,other):
        return Dual(self.val-other.val,self.deriv-other.deriv)
    #overload the * operator
    def __mul__(self,other):
        return Dual(self.val*other.val,self.val*other.deriv+other.val*self.deriv)
    #the negative operator simply takes the negative of both the derivative and primal value
    def __neg__(self):
        return Dual(-self.val,-self.deriv)
    def __truediv__(self,other):
        return Dual(self.val/other.val,(self.deriv*other.val-other.deriv*self.val)/(other.val)**2)
        '''
        TODO overload the division operator / 
        '''
        return None

In [30]:
def test_funct2(x,y):
    return x/y

x = 3
y = 2
print("Primal evaluation of function:",test_funct2(x,y))
print("Derivative with respect to x:",test_funct2(Dual(x,1),Dual(y,0)))
print("Derivative with respect to y:",test_funct2(Dual(x,0),Dual(y,1)))

Primal evaluation of function: 1.5
Derivative with respect to x: Value: 1.5 Derivative: 0.5
Derivative with respect to y: Value: 1.5 Derivative: -0.75


While the above class implementing Dual() numbers implements automatic differentiation satisfactorily, consider the number of function evaluations necessary to compute the gradient of the output with respect to each input, since this in the quantity of interest in machine learning.  Each input takes another evaluation of the function to determine the derivative of the output with respect to it.  Therefore, the run time of this method is again $\mathcal{O}(n)$, which is no better than that of numerical differentiation.

#### Reverse Mode

In contrast to the forward mode, the reverse mode runs through the graph, calculating the primal values and constructing the control flow graph before running back through the graph to calculate the derivative.  Therefore, unlike the dual number implementation of the forward mode, the graph must be explicitly constructed for the reverse mode.

After construction of the graph, in turn the output derivatives are calculated for each of the nodes in the graph.  These are termed adjoints in the automatic differention paper.  The adjoints are calculated by using the chain rule, taking into account both previously calculated adjoints and known derivatives of the symbolic functions stored on the edges between nodes.  The base case adjoint is the output derivative with respect to itself, which is $1$.  

For example, consider Figure 4 in the automatic differentiation paper.  After constructing the graph by running through the primal function, the base case adjoint can be determined using the chain rule $\overline{v_5} = \cfrac{\partial f}{\partial v_5} = 1$.  Then, the next layer of adjoints can be calculated 

$$\overline{v_4} = \cfrac{\partial f}{\partial v_4} = \cfrac{\partial f}{\partial v_5} \cfrac{\partial v_5}{\partial v_4} = 1 * 1 = 1$$

$$\overline{v_3} = \cfrac{\partial f}{\partial v_3} = \cfrac{\partial f}{\partial v_5} \cfrac{\partial v_5}{\partial v_3} = 1 * -1 = -1$$

Note that the non-primal derivatives are known, since the operation going from one node to the other has already been recorded, and that operation has a known derivative.  All adjoints can be calculated using this method.  However, note the special case of $\overline{v_{-1}}$ and $\overline{v_0}$ which have two outputs in contrast to the other nodes which only have a single output.  Using the chain rule, both outputs need to be considered as following.

$$\overline{v_{-1}} = \cfrac{\partial f}{\partial v_{-1}} = \cfrac{\partial f}{\partial v_1} \cfrac{\partial v_1}{\partial v_{-1}} + \cfrac{\partial f}{\partial v_2} \cfrac{\partial v_2}{\partial v_{-1}}$$

$$\overline{v_0} = \cfrac{\partial f}{\partial v_0} = \cfrac{\partial f}{\partial v_2} \cfrac{\partial v_2}{\partial v_0} + \cfrac{\partial f}{\partial v_3} \cfrac{\partial v_3}{\partial v_0}$$

Finally, the derivative of the output with respect to the inputs can be found.  However, unlike the forward method of automatic differentiation and numerical differentiation, this is done using only a constant number of evaluations of the function for each **output** and thus the run time to calculate the gradient is $\mathcal{O}(m)$ for m outputs.  This is why the reverse mode is used for back propagation in neural nets, as the output, in this case the loss function, is often only a scalar function of many input variables, sometimes millions.

#### Reverse Mode AD
We can easily create a dynamically created computational graph by identifying nodes in the graph with operations.  For example, we could have an "add" node that takes two arguments and produces a value, while saving the computed value and the parent(s).  Note that operations may be binary (as in add), unary (as in sin), or have no arguments at all (as in the case of an input value).  

In [34]:
# Input class (0-ary operation)
class FloatAD(object):
    def __init__(self,value,grad=0.):
        self.value = value
        self.grad = grad
        
    def backward(self,adjoint):
        self.grad += adjoint
    
# Addition of two numbers
class AddAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.x1 = None
        self.x2 = None
        self.value = None
    def forward(self,x1,x2):
        self.x1 = x1
        self.x2 = x2
        self.value = x1.value + x2.value
        return self
    
    def backward(self,adjoint):
        self.grad += adjoint
        self.x1.backward(self.grad)
        self.x2.backward(self.grad)    
           
# Multiplication of two numbers
class MultAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.x1 = None
        self.x2 = None
        self.value = None
    def forward(self,x1,x2):
        self.x1 = x1
        self.x2 = x2
        self.value = x1.value*x2.value
        return self
    
    def backward(self,adjoint):
        self.grad += adjoint
        self.x1.backward(self.grad*self.x2.value)
        self.x2.backward(self.grad*self.x1.value) 
        
# Sin of one number
class SinAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.value = None
        self.x = None
    def forward(self,x1):
        self.x = x1
        self.value = np.sin(x1.value)
        return self
    def backward(self,adjoint):
        self.grad += adjoint
        self.x.backward(self.grad*np.cos(self.x.value)) 
        
# Exponent of one number
class ExponentAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.value = None
        self.x = None
    def forward(self,x1,p):
        self.p = p
        self.x = x1
        self.value = x1.value**self.p
        return self
    def backward(self,adjoint):
        self.grad += adjoint
        self.x.backward(self.grad*self.p*self.x.value**(self.p-1))
      
# Evaluates the function sin(x1*(x2**2) + x1) and computes its gradient
# via reverse mode AD
x1 = FloatAD(1.0)
x2 = FloatAD(2.0)
x3 = ExponentAD().forward(x2,2)
v = MultAD().forward(x1,x3)
f = AddAD().forward(x1,v)

y = SinAD().forward(f)
y.backward(1.0)
print(f"reverse mode: {x1.grad}")

# Check correctness with the finite difference method
eps = 1e-6
x1 = FloatAD(1.0+eps)
x2 = FloatAD(2.0)
x3 = ExponentAD().forward(x2,2)
v = MultAD().forward(x1,x3)
f = AddAD().forward(x1,v)

y1 = SinAD().forward(f)
print(f"finite difference: {(y1.value - y.value)/eps}")

reverse mode: 1.4183109273161312
finite difference: 1.4183229137820774


**Augment the above code with a new operator to calculate either the unary natural log $\ln(x)$ operator or the binary division operator (and their derivatives)**.

In [47]:
# Input class (0-ary operation)
class FloatAD(object):
    def __init__(self,value,grad=0.):
        self.value = value
        self.grad = grad
        
    def backward(self,adjoint):
        self.grad += adjoint
    
# Addition of two numbers
class AddAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.x1 = None
        self.x2 = None
        self.value = None
    def forward(self,x1,x2):
        self.x1 = x1
        self.x2 = x2
        self.value = x1.value + x2.value
        return self
    
    def backward(self,adjoint):
        self.grad += adjoint
        self.x1.backward(self.grad)
        self.x2.backward(self.grad)    
           
# Multiplication of two numbers
class MultAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.x1 = None
        self.x2 = None
        self.value = None
    def forward(self,x1,x2):
        self.x1 = x1
        self.x2 = x2
        self.value = x1.value*x2.value
        return self
    
    def backward(self,adjoint):
        self.grad += adjoint
        self.x1.backward(self.grad*self.x2.value)
        self.x2.backward(self.grad*self.x1.value) 
        
# Sin of one number
class SinAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.value = None
        self.x = None
    def forward(self,x1):
        self.x = x1
        self.value = np.sin(x1.value)
        return self
    def backward(self,adjoint):
        self.grad += adjoint
        self.x.backward(self.grad*np.cos(self.x.value)) 
        
# Exponent of one number
class ExponentAD(object):
    def __init__(self,grad=0):
        self.grad = grad
        self.value = None
        self.x = None
    def forward(self,x1,p):
        self.p = p
        self.x = x1
        self.value = x1.value**self.p
        return self
    def backward(self,adjoint):
        self.grad += adjoint
        self.x.backward(self.grad*self.p*self.x.value**(self.p-1))
        
class NaturalLog(object):
    def __init__(self,grad = 0):
        self.grad = grad
        self.value = None 
        self.x = None
    def forward(self,x1):
        self.x = x1
        self.value = np.log(x1.value)
        return self
    def backward(self,adjoint):
        self.grad += adjoint
        self.x.backward(self.grad*1/(self.x.value))

# Evaluates the function ln(x) and computes its gradient
# via reverse mode AD
x1 = FloatAD(1.0)
x2 = FloatAD(2.0)
x3 = ExponentAD().forward(x2,2)
v = MultAD().forward(x1,x3)
f = AddAD().forward(x1,v)

y = NaturalLog().forward(f)
y.backward(1.0)
print(f"reverse mode: {x1.grad}")

# Check correctness with the finite difference method
eps = 1e-6
x1 = FloatAD(1.0+eps)
x2 = FloatAD(2.0)
x3 = ExponentAD().forward(x2,2)
v = MultAD().forward(x1,x3)
f = AddAD().forward(x1,v)

y1 = NaturalLog().forward(f)
print(f"finite difference: {(y1.value - y.value)/eps}")

reverse mode: 1.0
finite difference: 0.9999995000953277
